In [2]:
!pip install -q transformers einops accelerate langchain bitsandbytes
!pip install -qqq openai
!pip install -Uqqq chromadb

ERROR: Could not build wheels for chroma-hnswlib, which is required to install pyproject.toml-based projects


In [19]:
import os
import json
import textwrap

import langchain
# import chromadb
import transformers
import openai
import torch

from transformers import AutoTokenizer
from langchain import HuggingFacePipeline
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

In [23]:
import logging
logging.basicConfig(level=logging.DEBUG)

In [18]:
f = open("config.json")
config = json.load(f)
os.environ["OPENAI_API_KEY"] = config["key"]

In [25]:
model = "meta-llama/Llama-2-7b-chat-hf"
tokenizer = AutoTokenizer.from_pretrained(model)
logging.debug("Tokenizer loaded with model %s", model)

pipeline = transformers.pipeline(
    "text-generation",  # task
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
    max_length=1000,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id
)
logging.debug("Pipeline created with model %s", model)

class HuggingFacePipeline:
    def __init__(self, pipeline, model_kwargs):
        self.pipeline = pipeline
        self.model_kwargs = model_kwargs
        logging.debug("HuggingFacePipeline initialized with model_kwargs: %s", model_kwargs)

# LLM initialized in HuggingFace Pipeline wrapper
llm = HuggingFacePipeline(pipeline=pipeline, model_kwargs={'temperature': 0})
logging.debug("LLM initialized with temperature set to 0")

DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /meta-llama/Llama-2-7b-chat-hf/resolve/main/tokenizer_config.json HTTP/1.1" 200 0
DEBUG:root:Tokenizer loaded with model meta-llama/Llama-2-7b-chat-hf
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /meta-llama/Llama-2-7b-chat-hf/resolve/main/config.json HTTP/1.1" 200 0
DEBUG:filelock:Attempting to acquire lock 5306425008 on /Users/taylor/.cache/huggingface/hub/.locks/models--meta-llama--Llama-2-7b-chat-hf/66dec18c9f1705b9387d62f8485f4e7d871ca388718786737ed3c72dbfaac9fb.lock
DEBUG:filelock:Lock 5306425008 acquired on /Users/taylor/.cache/huggingface/hub/.locks/models--meta-llama--Llama-2-7b-chat-hf/66dec18c9f1705b9387d62f8485f4e7d871ca388718786737ed3c72dbfaac9fb.lock
/Users/taylor/Desktop/Grade 12/ATCS/atcs-final/.venv/lib/python3.12/site-packages/huggingface_hub/file_download.py:1003: UserWarning: Not enough free disk space to download the file. The expected file size is: 9976.58 MB. The target location /Users

KeyboardInterrupt: 